# Krauslesen: Erzeugen der TEIs

Wir gehen von dem mit dem Scraping-Notebook erzeugten JSON-Dump aus. Wir erzeugen daraus einzelne TEI Dateien und füllen den `teiHeader` soweit es geht aus. Auf die Verlinkung von Werken und Personen, Institutionen müssen wir einstweilig noch verzichten.

In [1]:
import json
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement
from datetime import datetime

In [2]:
#Register TEI Namespace
ET.register_namespace('',"http://www.tei-c.org/ns/1.0")

In [3]:
source_filename = "krauslesen_enriched_v2.json"

In [4]:
# Ausgangsdaten laden
f = open("krauslesen_enriched_v2.json", "r")
source_data = f.read()
f.close()

In [5]:
# als json parsen
data = json.loads(source_data)

In [6]:
# Ausgabe der Keys
#data.keys()
#Anzahl der Files: das Dictionary hat keys, die Anzahl sollte den Dokumenten entsprechen
print(len(data.keys()))

35


In [7]:
#Example : kkvo_1920-01-11
data["kkvo_1920-01-11"]

{'data_krausonline': {'date': '11.01.1920\n',
  'institutions': [{'link': 'https://www.kraus.wienbibliothek.at/content/wiener-konzerthaus',
    'name': 'Wiener Konzerthaus'},
   {'link': 'https://www.kraus.wienbibliothek.at/content/kinderausspeisung',
    'name': 'Kinderausspeisung'}],
  'krausonline_id': '7658',
  'krausonline_url': 'https://www.kraus.wienbibliothek.at/content/154-vorlesung-am-11011920',
  'objects': [],
  'persons': None,
  'place': 'Wien',
  'shortlink': 'https://www.kraus.wienbibliothek.at/node/7658',
  'summary_paragraphs': ['[Karl Kraus las im Mittleren Konzerthaussaal am] 11. Januar, ½3 Uhr:',
   'I. Lammasch und die Christen.',
   'Gerhart Hauptmann, »Die Weber« I. und II. Akt.',
   'II. »Die Weber«, IV. Akt.',
   'Gebet an die Sonne von Gibeon.',
   'Die Hälfte des Ertrags für die Kinderausspeisung.',
   '[Die Fackel 521-530, 01.1920, 98] - zitiert nach  Austrian Academy Corpus '],
  'title': '154. Vorlesung am 11.01.1920',
  'vorlesungsprogramm-transkript': [

Wir haben ein Template mit dem teiHeader erstellt, das mit dem Package `etree` eingelesen und befüllt wird.

In [8]:
#helper-function to generate a hash
import hashlib
def shorthash(textstring:str):
    # hash of string to be used in identifiers
    hash = hashlib.sha1(textstring.encode("UTF-8")).hexdigest()
    shorthash = hash[:8]
    return shorthash

In [9]:
#Funktion zur Generierung von einem TEI-File zu einem Dokument
def generate_TEI(doc_id):
    #namespace 
    ns = {"tei": "http://www.tei-c.org/ns/1.0"}
    
    # set filename
    filename = doc_id + ".xml"
    outfile = "tei/" + filename
    
    #Daten
    doc_data = data[doc_id]
    
    #Title without date: bare_title:
    if " am " in doc_data["data_krausonline"]["title"]:
        bare_title = doc_data["data_krausonline"]["title"].split(" am ")[0]
    else:
        #fallback
        bare_title = doc_data["data_krausonline"]["title"]
        
    #Date-Object:
    doc_date_obj = datetime.strptime(doc_data["data_krausonline"]["date"].strip(), "%d.%m.%Y")
    iso_date_string = doc_date_obj.strftime("%Y-%m-%d")
    
    
    #template
    tree = ET.parse("tei_template.xml")
    
    tree.getroot().attrib["xml:id"] = doc_id
    #tree.getroot().attrib["n"] = doc_id
    
    
    #Title
    new_title = "Karl Kraus: " + doc_data["data_krausonline"]["title"]
    tree.find("tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title",ns).text = new_title
    
    
    
    # -------- 
    # Krausonline source 
    # --------
    
    ko_source_xmlid = "krausonline_" + doc_data["data_krausonline"]["krausonline_id"]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']",ns).attrib["xml:id"] = ko_source_xmlid
    
    
    #Title in bibl[krausonline]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']/tei:title",ns).text = doc_data["data_krausonline"]["title"]
    
    #Author in bibl[krausonline]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']/tei:author",ns).text = "Karl Kraus"
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']/tei:author",ns).attrib["ref"] = "#P.Kraus"
    
    #IDs in bibl[krausonline]:
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']/tei:ref[@type='fulllink']",ns).text = doc_data["data_krausonline"]["krausonline_url"]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']/tei:ref[@type='fulllink']",ns).attrib["target"] = doc_data["data_krausonline"]["krausonline_url"]
    
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']/tei:ref[@type='shortlink']",ns).text = doc_data["data_krausonline"]["shortlink"]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']/tei:ref[@type='shortlink']",ns).attrib["target"] = doc_data["data_krausonline"]["shortlink"]
    
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#krausonline']/tei:idno[@type='ID']",ns).text = doc_data["data_krausonline"]["krausonline_id"]

    
    #Date in bibl is difficult, because we don't know the publication date..
    # What is the publication date of the Vorlesungsprogramm? Could put only a year there?
    
    
    # -------- 
    # Wienbibliothek source 
    # --------
    wienbib_source_xmlid = "wienbib_" + doc_data["data_wienbib"]["wienbib_id"]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']",ns).attrib["xml:id"] = wienbib_source_xmlid
    
    #Title
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:title",ns).text = doc_data["data_wienbib"]["titleHeader"]
    
    #Author
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:author",ns).text = "Karl Kraus"
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:author",ns).attrib["ref"] = "#P.Kraus"
    
    #Wienbib-ID
    #'wienbib_id': '1132421',
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:idno[@type='ID']",ns).text = doc_data["data_wienbib"]["wienbib_id"]
    
    
    #'wienbib_url': 'https://www.digital.wienbibliothek.at/wbrobv02/content/titleinfo/1132421'
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:ref[@type='fulllink']",ns).text = doc_data["data_wienbib"]["wienbib_url"]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:ref[@type='fulllink']",ns).attrib["target"] = doc_data["data_wienbib"]["wienbib_url"]
    
    #opac_url
    #'opac_url': 'https://permalink.obvsg.at/wbr/AC15960874',
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:ref[@type='opaclink']",ns).text = doc_data["data_wienbib"]["opac_url"]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:ref[@type='opaclink']",ns).attrib["target"] = doc_data["data_wienbib"]["opac_url"]
    
    #pdf-url
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:ref[@type='pdf']",ns).text = doc_data["data_wienbib"]["pdfdownload_url"].split("?")[0]
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#wienbib']/tei:ref[@type='pdf']",ns).attrib["target"] = doc_data["data_wienbib"]["pdfdownload_url"].split("?")[0]
    
    
    
    # -------- 
    # Fackel 
    # --------
    
    fackel_source_xmlid = "fackel_" + shorthash(doc_data["fackel"]["krausonline_bibl"])
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#fackel']",ns).attrib["xml:id"] = fackel_source_xmlid
    
    
    #krausonline_bibl
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#fackel']/tei:note[@source='#krausonline']",ns).attrib["source"] = "#" + ko_source_xmlid
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#fackel']/tei:note",ns).text = doc_data["fackel"]["krausonline_bibl"]
    
    #'issue': '521-530'
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#fackel']/tei:biblScope[@unit='issue']",ns).text = doc_data["fackel"]["issue"]
    #'pp': '98',
    tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#fackel']/tei:biblScope[@unit='page']",ns).text = doc_data["fackel"]["pp"]
    
    #'permalinks': ['https://fackel.oeaw.ac.at/f/521-530,098'],
    fackel_bibl_elem = tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:bibl[@source='#fackel']", ns)
    for permalink in doc_data["fackel"]["permalinks"]:
        fackel_ref_elem = SubElement(fackel_bibl_elem ,"ref")
        fackel_ref_elem.text = permalink 
        fackel_ref_elem.attrib["target"] = permalink
    
    
    # --------
    # msDesc
    # --------
    
    #msDesc
    msDesc_elem = tree.find("tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:msDesc", ns)
    #idnos have to be added manually, I am afraid..
    
    physDesc_elem = msDesc_elem.find("tei:physDesc",ns)
    ab_extent = SubElement(physDesc_elem, 'ab')
    ab_extent.attrib["type"] = "extent"
    ab_extent.text = doc_data["data_wienbib"]["extent"]
    
    ab_additional = SubElement(physDesc_elem, 'ab')
    ab_additional.text = doc_data["data_wienbib"]["anmerkung"]
    
    # --------
    # Abstracts
    # --------
    
    #ko-summary: 'summary_paragraphs'
    ko_abstract_elem = tree.find("tei:teiHeader/tei:profileDesc/tei:abstract[1]",ns)
    ko_abstract_elem.attrib["source"] = "#" + ko_source_xmlid
    for ko_abstract_p in doc_data["data_krausonline"]["summary_paragraphs"]:
        ko_abstract_p_elem = SubElement(ko_abstract_elem, "p")
        ko_abstract_p_elem.text = ko_abstract_p
    
    wienbib_abstract_elem = ET.Element("abstract")
    wienbib_abstract_elem.attrib["source"] = "#" + wienbib_source_xmlid
    
    wienbib_abstract_p_elem = SubElement(wienbib_abstract_elem, "p")
    wienbib_abstract_p_elem.text = doc_data["data_wienbib"]["zusammenfassung"]
    
        
    tree.find("tei:teiHeader/tei:profileDesc",ns).insert(1,wienbib_abstract_elem)
    
    # --------
    # Data on the event
    # --------
    tree.find("tei:teiHeader/tei:profileDesc/tei:settingDesc/tei:setting/tei:date",ns).text = doc_date_obj.strftime("%d.%m.%Y")
    tree.find("tei:teiHeader/tei:profileDesc/tei:settingDesc/tei:setting/tei:date",ns).attrib["when-iso"] = iso_date_string
    
    
    # --------
    # Transcription (kraus online)
    # --------
    krausonline_text_elem = tree.find("tei:text[@source='krausonline']",ns)
    krausonline_text_elem.attrib["source"] = "#" + ko_source_xmlid
    krausonline_text_body_elem = krausonline_text_elem.find("tei:body",ns)
    for transcript_p_text in doc_data["data_krausonline"]["vorlesungsprogramm-transkript"]:
        krausonline_text_body_p_elem = SubElement(krausonline_text_body_elem, "p")
        krausonline_text_body_p_elem.text = transcript_p_text
    
    
    #write
    tree.write(outfile, encoding='utf-8', xml_declaration = True )
    print("Exported " + outfile)
    

In [10]:
#test
#generate_TEI("kkvo_1920-01-11")

In [11]:
#loop over IDs
errors = []
for file_id in data.keys():
    try:
        generate_TEI(file_id)
    except:
        errors.append(file_id)
    

Exported tei/kkvo_1920-01-11.xml
Exported tei/kkvo_1920-01-22.xml
Exported tei/kkvo_1920-01-27.xml
Exported tei/kkvo_1920-01-29.xml
Exported tei/kkvo_1920-02-04.xml
Exported tei/kkvo_1920-02-15.xml
Exported tei/kkvo_1920-03-07.xml
Exported tei/kkvo_1920-03-14.xml
Exported tei/kkvo_1920-03-28.xml
Exported tei/kkvo_1920-04-06.xml
Exported tei/kkvo_1920-04-11.xml
Exported tei/kkvo_1920-04-18.xml
Exported tei/kkvo_1920-05-01.xml
Exported tei/kkvo_1920-05-09.xml
Exported tei/kkvo_1920-05-28.xml
Exported tei/kkvo_1920-05-29.xml
Exported tei/kkvo_1920-05-31.xml
Exported tei/kkvo_1920-06-02.xml
Exported tei/kkvo_1920-06-04.xml
Exported tei/kkvo_1920-06-11.xml
Exported tei/kkvo_1920-06-12.xml
Exported tei/kkvo_1920-06-14.xml
Exported tei/kkvo_1920-10-03.xml
Exported tei/kkvo_1920-10-09.xml
Exported tei/kkvo_1920-10-21.xml
Exported tei/kkvo_1920-11-01.xml
Exported tei/kkvo_1920-11-15.xml
Exported tei/kkvo_1920-11-28.xml
Exported tei/kkvo_1920-12-02.xml
Exported tei/kkvo_1920-12-08.xml
Exported t